# Get the player associated with each game

In [2]:
import pandas as pd
import os
import sys

In [3]:
# creates dictionary for the ABC code given the team_id
# {int: str} ex. {4: "TOR", 5: "WPG", ...}
teams_df = pd.read_csv("C:/Users/Owner/Desktop/cs stuff/Open Source/nhl-nn-sports-betting/data/team_info.csv")
team_dict = dict()
for index, row in teams_df.iterrows():
    team_dict[int(row['team_id'])] = str(row['abbreviation'])

In [4]:
# create a dictionary with both my_ids given a single game_id from the csv
# {str: list(str)} ex. {"1234567-5": ["TOR-2021-01-01", "WPG-2022-01-03"], ...}
from datetime import datetime


def convert_datetime(datetime_str):
    datetime_obj = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%SZ")
    date_str = datetime_obj.strftime("%Y-%m-%d")
    return date_str


game_df = pd.read_csv("C:/Users/Owner/Desktop/cs stuff/Open Source/nhl-nn-sports-betting/data/game.csv")
game_id_dict = dict()
for index, row in game_df.iterrows():
    try:
        date = str(convert_datetime(row['date_time_GMT']))
        home = str(team_dict[row['home_team_id']])
        away = str(team_dict[row['away_team_id']])
        their_id = int(row['game_id'])
        game_id_dict[their_id] = [f"{home}-{date}", f"{away}-{date}"]
    except:
        pass

In [5]:
# creates a dictionary to match the given game_id with my_ids
# {str: str} ex. {"1234567-5": "TOR-2021-01-01", "1234568-20": "WPG-2022-01-03", ...}
df = pd.read_csv("C:/Users/Owner/Desktop/cs stuff/Open Source/nhl-nn-sports-betting/data/game_skater_stats.csv")
my_id_dict = dict()
for index, row in df.iterrows():
    try:
        game_id = int(row['game_id'])
        team_id = int(row['team_id'])
        abrv = str(team_dict[row['team_id']])
        team1 = game_id_dict[row['game_id']][0]
        team2 = game_id_dict[row['game_id']][1]
        if team1[:3] == abrv:
            my_id_dict[f"{game_id}-{team_id}"] = team1
        else:
            my_id_dict[f"{game_id}-{team_id}"] = team2
    except:
        pass

In [9]:
from ordered_set import OrderedSet

player_set = OrderedSet()
for index, row in df.iterrows():
    if row['player_id'] in player_set:
        pass
    if int(row['timeOnIce']) > 750 and row['goals'] > 1:
        player_set.add(int(row['player_id']))

In [10]:
# create an empty list for each key in the previous dict
player_dict = dict()
for key in my_id_dict:
    player_dict[my_id_dict[key]] = []

In [11]:
# create dictionary that gives a list of player_ids for every player in a given game
# {str: list(int)} ex. {PHI-2016-10-19: [8468513, 8476906, ..., 8474668], ...}
for index, row in df.iterrows():
    try:
        game_id = int(row['game_id'])
        team_id = int(row['team_id'])
        player_id = int(row['player_id'])
        my_id =  my_id_dict[f"{game_id}-{team_id}"]
        
        if player_id in player_set:
            player_dict[my_id].append(player_id)
    except:
        pass

In [12]:
%store player_dict

Stored 'player_dict' (dict)
